# Unit 2 Assignment: Building a MoE Router

## Name : Shakirth Anisha
## SRN : PES2UG23CS927
## Section : 6 I

## Setup Environment


In [58]:
!pip install groq python-dotenv --quiet

In [59]:
import os
from dotenv import load_dotenv
from groq import Groq
import getpass

# Load environment variables from .env file
load_dotenv()

# Retrieve the Groq API key
if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your GROQ API Key: ")

GROQ_API_KEY = os.getenv("GROQ_API_KEY")

# Initialize the Groq client
client = Groq(
    api_key=GROQ_API_KEY,
)


### Define Your Experts

In [64]:
MODEL_CONFIG = {
    "technical": {
        "system_prompt": "You are a rigorous, very code focused, and precise technical support expert. You will provide clear, concise, and accurate solutions for coding issues, bugs, and technical queries. Ask for additional context whenever needed instead of providing example fixes.",
        "model": "moonshotai/kimi-k2-instruct"
    },
    "billing": {
        "system_prompt": "You are an empathetic, financially focused, policy driven billing support expert. You need to handle refund requests, subscription issues, and payment inquiries with care and accuracy. Refer to company policies where appropriate.",
        "model": "moonshotai/kimi-k2-instruct"
    },
    "general": {
        "system_prompt": "You are a friendly and helpful general customer support assistant. You will provide general information and engage in casual conversation.",
        "model": "moonshotai/kimi-k2-instruct"
    },
    "tool_use": {
        "system_prompt": "You are an expert assistant capable of using tools to fetch real time information. When asked for information that requires external data, use the appropriate tool. If no tool is available, state that you cannot provide real time data.",
        "model": "moonshotai/kimi-k2-instruct"
    }
}

def get_bitcoin_price_tool():
    import random
    return f"The current (mock) price of Bitcoin is ${random.randint(60000, 70000)} USD."


### The Router

In [65]:
def route_prompt(user_input):
    routing_prompt = f"""Classify the following user query into one of these categories:
- technical: for programming errors, code debugging, software issues, or anything requiring technical expertise.
- billing: for refunds, subscriptions, payment problems, or any financial transaction inquiries.
- general: for casual chat, general information not covered by other categories, or simple questions.
- tool_use: for queries requiring real-time data or external tools, such as current prices, weather forecasts, or other live information.

Return ONLY the category name (e.g., 'technical', 'billing', 'general', 'tool_use'), nothing else. If you are unsure, default to 'general'.

User Query: {user_input}
Category:"""

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant that classifies user queries.",
            },
            {
                "role": "user",
                "content": routing_prompt,
            }
        ],
        model="moonshotai/kimi-k2-instruct",
        temperature=0,
        max_tokens=10
    )
    category = chat_completion.choices[0].message.content.strip().lower()
    if category not in MODEL_CONFIG:
        return "general"
    return category


### The Orchestrator

In [66]:
def process_request(user_input):
    category = route_prompt(user_input)
    print(f"Routing '{user_input}' to the '{category}' expert.\n")

    if category == "tool_use":
        if "bitcoin" in user_input.lower() or "crypto" in user_input.lower():
            return get_bitcoin_price_tool()
        else:
            return "I don't have a tool for that specific request yet."

    expert_config = MODEL_CONFIG.get(category, MODEL_CONFIG["general"])
    system_prompt = expert_config["system_prompt"]
    model_name = expert_config["model"]

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": user_input,
            }
        ],
        model=model_name,
        temperature=0.7,
        max_tokens=500
    )

    return chat_completion.choices[0].message.content


### Demonstrate Functionality

In [67]:
print("------------------------------------------------ Example 1: Technical Query ------------------------------------------------ \n")
tech_query = "My Python script is throwing an IndexError on line 5. How do I fix it?"
tech_response = process_request(tech_query)
print(f"Technical Expert Response:\n{tech_response}\n")

print("\n------------------------------------------------ Example 2: Purchace Query ------------------------------------------------ \n")
billing_query = "I was charged twice for my subscription this month. Can I get a refund?"
billing_response = process_request(billing_query)
print(f"Billing Expert Response:\n{billing_response}\n")

print("\n------------------------------------------------ Example 3: General Query ------------------------------------------------ \n")
general_query = "Tell me a fun fact about AI."
general_response = process_request(general_query)
print(f"General Expert Response:\n{general_response}\n")

print("\n------------------------------------------------ Example 4: Tool Use Query ------------------------------------------------ \n")
tool_use_query_bitcoin = "What is the current price of Bitcoin?"
tool_use_response_bitcoin = process_request(tool_use_query_bitcoin)
print(f"Tool Use Expert Response (Bitcoin):\n{tool_use_response_bitcoin}\n---\n")

print("\n------------------------------------------------ Example 5: Tool Use Query (Out of scope) ------------------------------------------------ \n")
tool_use_query_weather = "What's the weather like in London?"
tool_use_response_weather = process_request(tool_use_query_weather)
print(f"Tool Use Expert Response (Weather):\n{tool_use_response_weather}\n---\n")


------------------------------------------------ Example 1: Technical Query ------------------------------------------------ 

Routing 'My Python script is throwing an IndexError on line 5. How do I fix it?' to the 'technical' expert.

Technical Expert Response:
IndexError on line 5 usually means you're trying to access an element in a sequence (list, tuple, string, array) that doesn't exist.

To fix it, share:

1. The sequence definition (list, array, etc.)
2. The line of code throwing the error
3. Any preceding logic that modifies the sequence

Let me see the line of code.


------------------------------------------------ Example 2: Purchace Query ------------------------------------------------ 

Routing 'I was charged twice for my subscription this month. Can I get a refund?' to the 'billing' expert.

Billing Expert Response:
I’m sorry you were billed twice—that’s frustrating, and we’ll get it straightened out quickly.

1. What I need to verify  
   • The e-mail address on the acc